# Working with Cloud (Remote) Back-ends
## So far
- We've learnt to create a synthetic dataset
- Train a logical function to make predictions from the synthetic dataset
- Test and develop an API server with Flask
- Deploy our prediction function to Heroku

## In this section
We will learn to interact with Google's Firebse and create a document collection **to demonstrate the usecase when our application needs to collect and store user data**.

## Setup and Authentication

In [1]:
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore

#### Point to Firebase credentials

In [2]:
cred = credentials.Certificate("../../configs/app_creds.json")

#### Initialize firebase client as admin

In [3]:
firebase_admin.initialize_app(cred,
    {
        "databaseURL": "https://save-nyc-demo.firebaseio.com/"
})

## Interfacing with your Firebase Collections

#### Load data to bootstrap

In [4]:
df = pd.read_csv("../../data/superhero_timesheets.csv")
df

,superhero,date,active_on_duty
0,Daredevil,2022-05-29,1
1,Deadpool,2022-05-29,1
2,Deadpool,2022-05-30,0
3,Deadpool,2022-05-31,0
4,Dr X,2022-05-31,0
5,Luke Cage,2022-05-30,0
6,Mr Fantastic,2022-05-30,0
7,Mr Fantastic,2022-05-31,1
8,Peeping Tom,2022-05-29,0
9,Peeping Tom,2022-05-30,1


#### Connect to your desired collection

In [5]:
db = firestore.client()
doc_ref = db.collection("superhero_timesheets")

#### Bootstrap all records

In [6]:
%%time
tmp = df.to_dict(orient="records")

list(map(lambda x: doc_ref.add(x), tmp))

CPU times: user 209 ms, sys: 108 ms, total: 317 ms
Wall time: 6.76 s


[(DatetimeWithNanoseconds(2022, 5, 31, 11, 17, 21, 824538, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x107bb5370>),
 (DatetimeWithNanoseconds(2022, 5, 31, 11, 17, 22, 65025, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x1307521f0>),
 (DatetimeWithNanoseconds(2022, 5, 31, 11, 17, 22, 597905, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x13075dd30>),
 (DatetimeWithNanoseconds(2022, 5, 31, 11, 17, 22, 882900, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x130752ca0>),
 (DatetimeWithNanoseconds(2022, 5, 31, 11, 17, 23, 417508, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x13075df70>),
 (DatetimeWithNanoseconds(2022, 5, 31, 11, 17, 23, 717888, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x130752b50>),
 (DatetimeWithNan

#### Query to fetch all records for today

In [7]:
today = pd.Timestamp.now().date()
res_0 = pd.DataFrame([
    doc.to_dict() for doc in db.collection(u"superhero_timesheets").where(u"date", u">=", str(today)).stream()
])

res_0

,active_on_duty,superhero,date
0,1,Mr Fantastic,2022-05-31
1,0,Deadpool,2022-05-31
2,1,Wolverine,2022-05-31
3,0,Dr X,2022-05-31
4,1,The Comedian,2022-05-31


#### Adding a record

In [11]:
%%time
doc_ref.add({
    "superhero": "Raphael",
    "active_on_duty": 1,
    "date": "2022-05-31"
})

CPU times: user 7.91 ms, sys: 7.75 ms, total: 15.7 ms
Wall time: 392 ms


(DatetimeWithNanoseconds(2022, 5, 31, 11, 18, 6, 234837, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x1306e0a60>)

In [12]:
res_1 = pd.DataFrame([
    doc.to_dict() for doc in db.collection(u"superhero_timesheets").where(u"date", u">=", str(today)).stream()
])

res_1

,superhero,active_on_duty,date
0,Raphael,1,2022-05-31
1,Mr Fantastic,1,2022-05-31
2,Deadpool,0,2022-05-31
3,Wolverine,1,2022-05-31
4,Leonardo,1,2022-05-31
5,Dr X,0,2022-05-31
6,The Comedian,1,2022-05-31
7,Donatello,1,2022-05-31
8,Michaelangelo,1,2022-05-31


#### Dropping a collection

In [13]:
%%time
def delete_collection(coll_ref, batch_size=200):
    docs = coll_ref.limit(batch_size).stream()
    deleted = 0

    for doc in docs:
        print(f'Deleting doc {doc.id} => {doc.to_dict()}')
        doc.reference.delete()
        deleted = deleted + 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)
    
delete_collection(doc_ref)

Deleting doc 1fht7jh4HBmDzAAdSnPn => {'date': '2022-05-29', 'superhero': 'Peeping Tom', 'active_on_duty': 0}
Deleting doc 3pSiTCpRkHjenrKn0nZm => {'active_on_duty': 0, 'date': '2022-05-30', 'superhero': 'Mr Fantastic'}
Deleting doc 4HxCsLyX8bKLErWoQ7Bo => {'active_on_duty': 1, 'superhero': 'Raphael', 'date': '2022-05-31'}
Deleting doc 6o7Yht6e9TPS7yYaJuIr => {'superhero': 'Mr Fantastic', 'active_on_duty': 1, 'date': '2022-05-31'}
Deleting doc AlrXwf4lsMj4fsMSdktN => {'superhero': 'Deadpool', 'date': '2022-05-31', 'active_on_duty': 0}
Deleting doc D4YMalQ8V6m6UcGQFkpI => {'superhero': 'Peeping Tom', 'date': '2022-05-30', 'active_on_duty': 1}
Deleting doc DTlPHyHLewvaSZqLhCqi => {'date': '2022-05-30', 'active_on_duty': 1, 'superhero': 'Spiderman'}
Deleting doc Oo7wJLe57NxQLy2b16vA => {'date': '2022-05-29', 'active_on_duty': 1, 'superhero': 'Daredevil'}
Deleting doc Pbvp4iSQqXjLYAXyU69B => {'active_on_duty': 0, 'superhero': 'Luke Cage', 'date': '2022-05-30'}
Deleting doc TpArgPjqmA1Csw0gs

## Outro
Thank you for following this workshop all the way to the ends. All credits to [Firebase Docs](https://firebase.google.com/docs) for code snippets used in this notebook.